In [ ]:
import os
import pandas as pd

def carregar_idxstats(caminho):
    df = pd.read_csv(caminho, sep='\t')
    df = df[['CDS', 'RPKM']]  
    return df

def unificar_dados(idxstats_dir, saida):
    todas_amostras = []

    for arquivo in os.listdir(idxstats_dir):
        if arquivo.endswith(".idxstats.txt"):
            amostra = arquivo.split(".")[0]
            caminho_idx = os.path.join(idxstats_dir, arquivo)

            df_idx = carregar_idxstats(caminho_idx)
            df_idx['Amostra'] = amostra

            todas_amostras.append(df_idx)

    df_final = pd.concat(todas_amostras, ignore_index=True)
    df_final.to_csv(saida, sep='\t', index=False)

    print(f"Dados unificados salvos em: {saida}")

# Caminhos
dir_idxstats = "/home/gabrielacastilho/Jo/idxstats"
arquivo_saida = "/home/gabrielacastilho/Jo/idxstats/dados_unificados_idxstats_Jo.tsv"

# Rodar
unificar_dados(dir_idxstats, arquivo_saida)


In [ ]:
import pandas as pd

arquivo_unificado = "/home/gabrielacastilho/Jo/idxstats/dados_unificados_idxstats_Jo.tsv"
arquivo_amostras = "/home/gabrielacastilho/Jo/amostras_formatadas_Jo.csv"
arquivo_saida = "/home/gabrielacastilho/Jo/idxstats/Jo_dados_idxstats_com_grupo.tsv"

# Carregar cara-crachá (pequeno, ok na RAM)
df_amostras = pd.read_csv(arquivo_amostras)
df_amostras.columns = df_amostras.columns.str.strip()

primeiro = True

# Ler o arquivo grande em chunks
for chunk in pd.read_csv(
    arquivo_unificado,
    sep='\t',
    chunksize=1_000_000
):
    df_chunk = chunk.merge(
        df_amostras[['Amostra', 'Grupo']],
        on='Amostra',
        how='left'
    )

    df_chunk.to_csv(
        arquivo_saida,
        sep='\t',
        index=False,
        mode='w' if primeiro else 'a',
        header=primeiro
    )

    primeiro = False
    print(f"[OK] chunk processado ({len(df_chunk)} linhas)")

print("Finalizado sem morrer 🙏")

In [ ]:
#Sanity check: is there doubled CDS per sample? 
import pandas as pd 
df = pd.read_csv("/home/gabrielacastilho/Jo/idxstats/Jo_dados_idxstats_com_grupo.tsv", sep='\t') 
duplicados = df[df.duplicated(subset=['CDS', 'Amostra'], keep=False)] 
print(f"Número de linhas duplicadas: {duplicados.shape[0]}") 